# EXP 14. Movielens 영화 추천

## 0. 들어가며

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.



### 0.1 프로젝트 목표
***
* MF모델, AlternatingLeastSquares모델에 대해 이해하기
* 아이템, 유저/유사도, 기여도에 대해 이해하기

### 0.2 루브릭
***

|**평가문항**|**상세기준**|
|------------|-------------|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

### 0.2 Import package

In [1]:
# 커널정보 3.7
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1. 데이터 준비와 전처리
***
Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path = 'D:/project/aiffel_exp/data/exp14/project/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::',
                      names=ratings_cols, engine='python', encoding="ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='D:/project/aiffel_exp/data/exp14/project/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [7]:
# movies와 ratings을 합침
data = pd.merge(movies, ratings)
cols = cols + ratings_cols 
data.head()

,movie_id,title,genre,user_id,counts,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [8]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['movie_id', 'title', 'user_id', 'counts']
data = data[using_cols]
data.head()

,movie_id,title,user_id,counts
0,1,Toy Story (1995),1,5
1,1,Toy Story (1995),6,4
2,1,Toy Story (1995),8,4
3,1,Toy Story (1995),9,5
4,1,Toy Story (1995),10,5


In [9]:
# 검색을 쉽게 하기 위해 타이틀 문자열을 소문자로 바꿔줍시다.
data['title'] = data['title'].str.lower() 
data.head()

,movie_id,title,user_id,counts
0,1,toy story (1995),1,5
1,1,toy story (1995),6,4
2,1,toy story (1995),8,4
3,1,toy story (1995),9,5
4,1,toy story (1995),10,5


In [10]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,movie_id,title,user_id,counts
0,1,toy story (1995),1,5
19773,48,pocahontas (1995),1,5
35123,150,apollo 13 (1995),1,5
54895,260,star wars: episode iv - a new hope (1977),1,4
116019,527,schindler's list (1993),1,5
119008,531,"secret garden, the (1993)",1,4
130190,588,aladdin (1992),1,4
138999,594,snow white and the seven dwarfs (1937),1,4
139706,595,beauty and the beast (1991),1,5
142605,608,fargo (1996),1,4


## 2. 분석해 봅시다.
***
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [11]:
# 유니크한 영화 수
data['movie_id'].nunique()

3628

In [12]:
# 유니크한 유저 수
data['user_id'].nunique()

6039

In [13]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(20)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [14]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

와우, 1968편

In [15]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [16]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['matrix, the (1999)',
               'jurassic park (1993)',
               'sixth sense, the (1999)',
               'fargo (1996)',
               'braveheart (1995)']

# 'jby'이라는 user_id가 위 영화를 평점 5점씩 줬다고 가정
my_playlist = pd.DataFrame({'user_id': ['jby']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['jby']})['user_id'].any():  # user_id에 'jby'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,user_id,counts
836473,3952.0,"contender, the (2000)",5682,3
836474,3952.0,"contender, the (2000)",5812,4
836475,3952.0,"contender, the (2000)",5831,3
836476,3952.0,"contender, the (2000)",5837,4
836477,3952.0,"contender, the (2000)",5998,4
0,NaN,"matrix, the (1999)",jby,5
1,NaN,jurassic park (1993),jby,5
2,NaN,"sixth sense, the (1999)",jby,5
3,NaN,fargo (1996),jby,5
4,NaN,braveheart (1995),jby,5


In [17]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jby'])    # 6038명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 

6039


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
print(len(temp_user_data))
print(len(data))
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie(title) 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
print(len(temp_user_data))
print(len(data))
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

836483
836483
user_id column indexing OK!!
836483
836483
title column indexing OK!!


,movie_id,title,user_id,counts
0,1.0,0,0,5
1,1.0,0,1,4
2,1.0,0,2,4
3,1.0,0,3,5
4,1.0,0,4,5
...,...,...,...,...
0,NaN,2325,6039,5
1,NaN,462,6039,5
2,NaN,2507,6039,5
3,NaN,587,6039,5


## 4. CSR matrix를 직접 만들어 봅시다.

In [20]:
# 실습 위에 설명보고 이해해서 만들어보기
num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [21]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
FACTORS = 100
ITTER = 30

als_model = AlternatingLeastSquares(factors=FACTORS,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=ITTER,
                                    dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

100%|██████████| 30/30 [00:16<00:00,  1.77it/s]


In [31]:
# 모델 학습 후 벡터 생성과 곱 확인
jby, matrix = user_to_idx['jby'], movie_to_idx['matrix, the (1999)']
jby_vector, matrix_vector = als_model.user_factors[jby], als_model.item_factors[matrix]

print('슝=3')

슝=3


In [32]:
jby_vector

array([ 0.04512807, -0.64097923,  0.39225993,  0.03780821,  0.16053276,
        0.22409485,  0.44845563, -0.6605327 , -0.23602848, -1.0473623 ,
       -0.04746384,  0.38587812, -0.15147588,  0.01836262,  0.62070924,
        0.42445302, -0.4846172 , -0.24358752, -0.07343699, -0.055594  ,
        0.50525486, -0.79153186,  0.56302685,  0.32517037,  0.1991601 ,
        0.1701948 , -0.05580804, -0.3493529 ,  0.41086218, -0.01977657,
       -0.07523528, -0.16283597, -0.15748914,  0.6087057 , -0.25158232,
        0.3827015 ,  0.4831052 , -0.92160654, -0.26556683,  0.11990332,
        0.16852069,  0.08369899,  0.19729726, -0.09143362, -0.51526755,
        0.6471554 ,  0.04040889,  0.64743984,  0.2802972 ,  0.15040325,
        0.01933583,  0.19974548, -0.01609626,  0.07513096,  0.60655993,
        0.12955502, -0.5778465 ,  0.13701862,  0.02333194, -0.2031191 ,
        0.5855951 , -0.8440647 ,  0.66847897, -0.08179139, -0.550228  ,
        0.19647895, -0.40039435, -0.738836  ,  0.6483384 , -0.54

In [33]:
matrix_vector

array([ 0.01040251, -0.03954237,  0.01599171,  0.01699576, -0.00448875,
        0.01859375,  0.00770101, -0.01587947,  0.00951681, -0.01476649,
        0.01698   ,  0.00115889,  0.01085224,  0.0108203 ,  0.0100208 ,
        0.01389251, -0.00948344,  0.00781914,  0.00363766,  0.03018957,
        0.01022542, -0.03587609,  0.04729893,  0.01615578,  0.01418382,
        0.00580582, -0.00707515, -0.00294035,  0.04275889, -0.01242377,
        0.01053252,  0.00074082,  0.00643579,  0.02084895,  0.0019383 ,
        0.01837972,  0.00710576, -0.0309607 , -0.03473994,  0.00893925,
        0.02060835,  0.01614938, -0.01354734, -0.02631439, -0.01350984,
        0.02366926,  0.02203807,  0.02703755, -0.00369651,  0.03816856,
       -0.02165268,  0.02220775,  0.02777162,  0.01929121,  0.01651498,
        0.00871326,  0.00099825, -0.00581705,  0.02088829, -0.00154535,
        0.0250308 ,  0.01164708,  0.05829848, -0.00531552, -0.00466894,
       -0.00816062, -0.01717138, -0.00948037,  0.0213059 , -0.04

In [35]:
# 두 벡터를 곱해 1에 가까운 수가 나올 수록 학습이 잘 된 것이다.
np.dot(jby_vector, matrix_vector)

0.6099028

In [36]:
# Implicit AlternatingLeastSquares 모델의 선언
FACTORS = 200
ITTER = 30

als_model = AlternatingLeastSquares(factors=FACTORS,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=ITTER,
                                    dtype=np.float32)

In [37]:
# 모델 훈련
als_model.fit(csr_data_transpose)

100%|██████████| 30/30 [00:20<00:00,  1.50it/s]


In [43]:
# 모델 학습 후 벡터 생성과 곱 확인
jby, matrix = user_to_idx['jby'], movie_to_idx['matrix, the (1999)']
jby_vector, matrix_vector = als_model.user_factors[jby], als_model.item_factors[matrix]

jby_vector
matrix_vector
np.dot(jby_vector, matrix_vector)

0.6717176

In [44]:
# Implicit AlternatingLeastSquares 모델의 선언
FACTORS = 200
ITTER = 50

als_model = AlternatingLeastSquares(factors=FACTORS,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=ITTER,
                                    dtype=np.float32)

als_model.fit(csr_data_transpose)

100%|██████████| 50/50 [00:35<00:00,  1.43it/s]


In [45]:
# 모델 학습 후 벡터 생성과 곱 확인
jby, matrix = user_to_idx['jby'], movie_to_idx['matrix, the (1999)']
jby_vector, matrix_vector = als_model.user_factors[jby], als_model.item_factors[matrix]

jby_vector
matrix_vector
np.dot(jby_vector, matrix_vector)

0.6812681

In [46]:
# Implicit AlternatingLeastSquares 모델의 선언
FACTORS = 400
ITTER = 50

als_model = AlternatingLeastSquares(factors=FACTORS,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=ITTER,
                                    dtype=np.float32)

als_model.fit(csr_data_transpose)

100%|██████████| 50/50 [00:42<00:00,  1.19it/s]


In [47]:
# 모델 학습 후 벡터 생성과 곱 확인
jby, matrix = user_to_idx['jby'], movie_to_idx['matrix, the (1999)']
jby_vector, matrix_vector = als_model.user_factors[jby], als_model.item_factors[matrix]

jby_vector
matrix_vector
np.dot(jby_vector, matrix_vector)

0.89807975

FACTORS = 400, ITTER=50에서 0.898까지 상승했다.  
FACTORS 값을 크게 줄수록 1에 가까워지나 과적합이 우려되어 0.898 값으로 진행

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 6.1 내가 선호하는 영화 중 하나 선호도 파악

In [57]:
# 선호하는 영화 쥬라기 파크에 대한 선호도 파악
jp = movie_to_idx['jurassic park (1993)']
jp_vector = als_model.item_factors[jp]
np.dot(jby_vector, jp_vector)

0.9349424

### 6.2 그 외의 영화 선호도 파악

In [59]:
# pot o' gold 에 대한 선호도 0.004
pot = movie_to_idx["pot o' gold (1941)"]
pot_vector = als_model.item_factors[pot]
np.dot(jby_vector, pot_vector)

0.0044613252

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### 7.1 matrix, the (1999) 와 비슷한 영화 추천

In [62]:
favorite_movie = 'matrix, the (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(2325, 1.0),
 (569, 0.24768),
 (3334, 0.2409196),
 (1386, 0.24063696),
 (1004, 0.23742867),
 (2211, 0.23713905),
 (2505, 0.23699114),
 (2569, 0.23624307),
 (2330, 0.23594928),
 (2950, 0.23591046),
 (2385, 0.23574612),
 (57, 0.23552635),
 (719, 0.23534372),
 (609, 0.2344041),
 (2655, 0.23429614)]

In [63]:
#movie_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'running free (2000)',
 'second jungle book: mowgli & baloo, the (1997)',
 'search for one-eye jimmy, the (1996)',
 'gate ii: trespassers, the (1990)',
 'gambler, the (a játékos) (1997)',
 'on the ropes (1999)',
 'love, etc. (1996)',
 'smashing time (1967)',
 "mummy's ghost, the (1944)",
 'confessional, the (le confessionnal) (1995)',
 'babyfever (1994)',
 'theodore rex (1995)',
 'molly (1999)']

터미네이터 말고는 모르는 영화들이다.

### 7.2 jurassic park (1993) 와 비슷한 영화 추천

In [66]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [67]:
get_similar_movie('jurassic park (1993)')

['jurassic park (1993)',
 'midaq alley (callejón de los milagros, el) (1995)',
 "pot o' gold (1941)",
 'reckless (1995)',
 'designated mourner, the (1997)',
 'schlafes bruder (brother of sleep) (1995)',
 'men in black (1997)',
 'cabaret balkan (bure baruta) (1998)',
 'loser (1991)',
 'lost world: jurassic park, the (1997)']

In [68]:
get_similar_movie('sixth sense, the (1999)')

['sixth sense, the (1999)',
 'communion (a.k.a. alice, sweet alice/holy terror) (1977)',
 'dry cleaning (nettoyage à sec) (1997)',
 'brother minister: the assassination of malcolm x (1994)',
 "heaven's burning (1997)",
 'match, the (1999)',
 'boys life 2 (1997)',
 'skipped parts (2000)',
 'heaven (1998)',
 'project moon base (1953)']

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. 
`filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [71]:
user = user_to_idx['jby']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(569, 0.19647154),
 (1419, 0.13882735),
 (1233, 0.1265754),
 (1448, 0.11882388),
 (49, 0.11603098),
 (1540, 0.10965714),
 (439, 0.10268031),
 (513, 0.09996404),
 (1810, 0.099620655),
 (2885, 0.09797871),
 (684, 0.09241292),
 (2285, 0.09207582),
 (1097, 0.09064489),
 (573, 0.08897412),
 (2037, 0.08860007),
 (2456, 0.08836229),
 (2659, 0.084173344),
 (1024, 0.0841715),
 (1248, 0.08307285),
 (1166, 0.08184196)]

In [72]:
[idx_to_movie[i[0]] for i in movie_recommended]

['terminator 2: judgment day (1991)',
 'men in black (1997)',
 'sling blade (1996)',
 'hunt for red october, the (1990)',
 'usual suspects, the (1995)',
 'titanic (1997)',
 'fugitive, the (1993)',
 'secret garden, the (1993)',
 'saving private ryan (1998)',
 'bicentennial man (1999)',
 'rock, the (1996)',
 'planet of the apes (1968)',
 'goodfellas (1990)',
 'silence of the lambs, the (1991)',
 'rush hour (1998)',
 'blair witch project, the (1999)',
 "ferris bueller's day off (1986)",
 'return of the pink panther, the (1974)',
 'star trek: the wrath of khan (1982)',
 'big sleep, the (1946)']

### 8.1 왜 해당 영화를 추천했는지 살펴보기

In [74]:
mib = movie_to_idx['men in black (1997)']
explain = als_model.explain(user, csr_data, itemid=mib)

In [75]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('jurassic park (1993)', 0.07577662833988807),
 ('braveheart (1995)', 0.02619446964617387),
 ('matrix, the (1999)', 0.017277678510153067),
 ('sixth sense, the (1999)', 0.009842197419857989),
 ('fargo (1996)', 0.008655846244871963)]

In [76]:
tanic = movie_to_idx['titanic (1997)']
explain = als_model.explain(user, csr_data, itemid=tanic)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]


[('jurassic park (1993)', 0.02918949783002219),
 ('fargo (1996)', 0.028031489948308654),
 ('sixth sense, the (1999)', 0.0246887332133778),
 ('matrix, the (1999)', 0.020982325414003137),
 ('braveheart (1995)', 0.006122793809210003)]

## 느낀점

* 평소 궁금했던 추천 시스템에 관한 이론과 모델 등을 배우고 실습해 볼 수 있어서 재밌었다.
* csv가 두개라 처음에 좀 헤맸다. 엑셀이었다면 당연하게 하나의 sheet로 합쳐서 데이터를 정리했을텐데, 코딩 화면은 아직도 낯설다.
* 외국 영화는 유명한 몇 편정도만 챙겨보는 수준이라, 유사도에 대한 결과로 나온 영화들 중 아는 영화가 너무 적어 분석 및 판단하기가 어려웠다.
* 로컬에서 노드를 따라할 때 model fit의 결과가 이상해져 오류가 발생했다. `implicit`의 버전을 노드 버전인 0.4.8로 동일하게 맞춰주자 해결됐다.
* 모델의 학습이 굉장히 빨라 간만에 시원했던 exp였다. implicit의 gpu옵션은 `True`로 설정하니 오류가 뜨던데 해결하지 못했다.
* factors와 itter를 조절해 모델 훈련의 정확도를 계속 높일 수 있었다. 이 모델은 과적합이 발생하는 경우에는 어떤 처리를 하는지 궁금하다.